In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# 导入待分析数据

file_path = Path.cwd() / "fzu_jwch_notice.csv"

df = pd.read_csv(file_path)


## 1.**统计通知发布者次数和占比**

In [ ]:

#通知总数量
tot = len(df)

notifierGroup = df.groupby("notifier").agg(
    通知部门=('notifier','first'),
    通知数量=('notifier','size'),
    占比=('notifier',lambda x :str(round(len(x)/tot *100,2)))
).reset_index(drop= True)

display(notifierGroup.style.set_properties(**{'text-align': 'left'}))

plt.rcParams['font.sans-serif'] = ['SimHei']

ax = notifierGroup.plot(kind='bar',x='通知部门',y='通知数量',xlabel='通知部门',ylabel='通知数量',title="各部门通知数量")

ax.bar_label(
    container=ax.containers[0]
)


ax = notifierGroup.plot(
    kind='pie',
    title= "各部门通知数量占比",
    y='通知数量',
    labels = notifierGroup['通知部门'],
    autopct = '%1.1f%%',
    figsize=(8,6)
)

ax.set_aspect('equal', adjustable='datalim')

## 2.**统计通知发布者和附件下载次数的关系**

In [ ]:
import re

def getDownloadTimes(x):
    a = re.findall(r"(?<=')\d+(?=')",x)
    ans = 0
    for i in a:
        ans += int(i)
    return ans

#把列表转换为数字
df["download_counts"] = df["download_times"].apply(getDownloadTimes)

notifierGroup2 = df.groupby("notifier").agg(
    通知人 = ("notifier","first"),
    附件下载总次数 = ("download_counts","sum")
).reset_index(drop=True)

display(notifierGroup2.style.set_properties(**{'text-align': 'left'}))

ax = notifierGroup2.plot(
    kind='bar',
    x = '通知人',
    y = '附件下载总次数',
    ylabel='附件下载次数',
    title= '通知发布者和附件下载次数的关系'
)

ax.bar_label(
    ax.containers[0]
);


# 3.通知发布的日期统计

1. 统计每天发布的通知数量

In [ ]:
dateGroupDaily = df.groupby("date").agg(
    日期 = ("date","first"),
    通知数 = ("date","size")
).reset_index(drop=True)
print(dateGroupDaily.to_string())

2. 统计每月发布的通知数量(更容易看出通知数量在一学期中的分布规律)

In [ ]:
# 转换日期数据

df["date"] = pd.to_datetime(df["date"])

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

# 按照月份分组
dateGroupMonthly = df.groupby(["year","month"]).agg(
    通知数量 = ("date","size")
)
dateGroupMonthly.plot(kind='bar')
display(dateGroupMonthly)

> 可以看出在4,5,6月通知最多,暑假的时候(7,8月份)通知较少

# 4. 通知发布人和附件平均下载次数的关系

In [ ]:
# 获取每个通知的附件平均下载次数

df["average_download_times"] =df[df["attachment_cnt"] !=0 ]["download_counts"]/ df[df["attachment_cnt"] !=0 ]["attachment_cnt"]


# 填充
df.fillna(value=0,inplace=True)

groupDownloadTime = df.groupby("notifier").agg(
    通知发布人 = ("notifier","first"),
    平均文件下载次数 = ("average_download_times",lambda x:x[x!=0].mean())
).reset_index(drop=True)

groupDownloadTime.fillna(value=0,inplace=True)

display(groupDownloadTime.style.format(precision=2))